# Instructions:

Please follow the steps below, before running the script:


1.  Navigate to `Runtime`>`Change runtime type` and select **GPU** as Hardware accelerator 
2.   Download this as .zip 
1.   Choose `Files` from the side menu
2.   Upload the downloaded .zip file
3.   Run the following cells in order





In [0]:
!unzip dataset.zip
!pip install biopython

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import os
import numpy as np
import pandas as pd
import evaluation as evaal
import preprocessing as prp
import functions as func
from datetime import datetime
from tensorflow import keras
#print(keras.__version__)
from keras.preprocessing.sequence import pad_sequences

TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


### Preprocessing

In [0]:
#@title Please choose the algorithm that you want to use: { form-width: "250px", display-mode: "both" }
algorithm = "SVM" #@param ["SVM", "BiLSTM"]



# ========== Importing the data

X, y = prp.import_data(algorithm)

# =========================== Preprocessing ===============================

# ========== Ordinal encoding

amino_codes = ['0', 'A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P', 'S', 'R', 'T', 'W', 'V', 'Y']
non_amino_letters = ['B', 'J', 'O', 'U', 'X', 'Z']

amino_mapping = prp.create_mapping(amino_codes)
 
X['mapped_seq'] = prp.integer_encoding(X['seq'], amino_mapping) 

# ========== Sequence padding

X_pad = pad_sequences(X['mapped_seq'], maxlen=3800, padding='post', truncating='post')

# ===================== Dimensionality reduction

# =========== RFE
#reduced_X = prp.dim_reduction_RFE(X_pad, y, 100)
# If you are using new dataset, run the above line and comment the lines in the below
# Otherwise, in order to save time the optimal features have been saved in a file and will be read from the file (features_100.csv)

feat_support = pd.read_csv('features_100.csv')

X = pd.DataFrame(data=X_pad)
reduced_X = pd.DataFrame()
c = 0
for index, r in feat_support.iterrows():
    if(r[0] == True):
        reduced_X.loc[:,c] = X.iloc[:, index]
        c+=1

if (algorithm == "BiLSTM"):
    # Add the generated data (in the same order of their labels as they were added in preprocessing.py) at the end of X
    reduced_X = prp.add_synthetics("dataset/cytoplasmiccytoplasmicmembrane_synthetic(cyt&cm).txt", reduced_X)
    reduced_X = prp.add_synthetics("dataset/periplasmiccytoplasmicmembrane_synthetic(per_cm&per).txt", reduced_X)
    reduced_X = prp.add_synthetics("dataset/outermembraneextracellular_synthetic(om_ext&ext).txt", reduced_X)



### Machine Learning models

In [3]:
#@title Please choose the run type: { form-width: "250px" }
run_type = "10_fold_regular" #@param ["10_fold_regular", "10_fold_grand_mean"]
#@markdown * 10_fold_grand_mean will run 10_fold_regular for 30 times.
# ========================= Machine learning models ======================

if (algorithm == "SVM"):
    # ========== 1. Support Vector Machine (SVM) ==========
    
    # ==========  Hyperparameter tuning    
    #svm_params = evaal.SVM_tuning(reduced_X, y)
    svm_params = {'C': 50, 'gamma': 0.0001, 'kernel': 'rbf'}
    
    # ==========  Model evaluation using 10-fold cross-validation
#    evaal.svm_eval(reduced_X, y, svm_params, run_type)
    evaal.svm_eval(reduced_X, y, svm_params, run_type)

elif (algorithm == "BiLSTM"):
    # ========== 2. Deep learning ==========
    
    # ========== One-hot encoding    
    X_ohe, y_ohe = prp.one_hot_encoding(reduced_X, y)    
    
    # ========== Bidirectional Long short-term memory networks (Bi-LSTMs)
    # ========== Model evaluation using 10-fold Cross-validation
#    evaal.lstm_eval(reduced_X, y_ohe, run_type)
    evaal.lstm_eval(reduced_X, y_ohe, run_type)



Regular 10-fold cross-validation Bi-LSTM
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 10917 samples, validate on 1213 samples
Epoch 1/50
10917/10917 [==============================] - 7s 642us/sample - loss: 1.7946 - precision: 0.3667 - val_loss: 1.3673 - val_precision: 0.8174
Epoch 2/50
10917/10917 [==============================] - 1s 69us/sample - loss: 1.0397 - precision: 0.7649 - val_loss: 0.9064 - val_precision: 0.7659
Epoch 3/50
10917/10917 [==============================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 10917 samples, validate on 1213 samples
Epoch 1/50
10917/10917 [==============================] - 1s 105us/sample - loss: 1.7173 - precision: 0.4609 - val_loss: 1.3588 - val_precision: 0.5895
Epoch 2/50
10917/10917 [==============================] - 1s 68us/sample - loss: 1.1368 - precision: 0.7716 - val_loss: 0.9062 - val_precision: 0.8355
Epoch 3/50
10917/10917 [==============================] - 1s 68us/sample - loss: 0.8071 - precision: 0.8214 - val_loss: 0.7080 - val_precision: 0.8172
Epoch 4/50
10917/10917 [==============================] - 1s 67us/sample - loss: 0.6803 - precision: 0.8318 - val_loss: 0.6466 - val_precision: 0.8421
Epoch 5/50
10917/10917 [==============================] - 1s 68us/sample - loss: 0.6160 - precision: 0.8449 - val_loss: 0.5870 - val_precision: 0.8688
Epoch 6/50
10917/10917 [==============================] - 1s 67us/sample - loss: 0.5638 - precision: 0.8728 - val_loss: 0.5469 - val_precision: 0.8849
Epoch 7/50
10917/10917 [====================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 10917 samples, validate on 1213 samples
Epoch 1/50
10917/10917 [==============================] - 1s 114us/sample - loss: 1.8039 - precision: 0.3502 - val_loss: 1.5839 - val_precision: 0.8402
Epoch 2/50
10917/10917 [==============================] - 1s 69us/sample - loss: 1.1013 - precision: 0.7778 - val_loss: 0.8381 - val_precision: 0.8004
Epoch 3/50
10917/10917 [==============================] - 1s 69us/sample - loss: 0.7678 - precision: 0.8205 - val_loss: 0.7271 - val_precision: 0.8528
Epoch 4/50
10917/10917 [==============================] - 1s 68us/sample - loss: 0.6718 - precision: 0.8416 - val_loss: 0.6237 - val_precision: 0.8651
Epoch 5/50
10917/10917 [==============================] - 1s 68us/sample - loss: 0.6160 - precision: 0.8597 - val_loss: 0.6280 - val_precision: 0.8622
Epoch 6/50
10917/10917 [==============================] - 1s 69us/sample - loss: 0.5834 - precision: 0.8691 - val_loss: 0.6026 - val_precision: 0.8579
Epoch 7/50
10917/10917 [====================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 10917 samples, validate on 1213 samples
Epoch 1/50
10917/10917 [==============================] - 2s 145us/sample - loss: 1.7798 - precision: 0.4330 - val_loss: 1.3440 - val_precision: 0.7200
Epoch 2/50
10917/10917 [==============================] - 1s 71us/sample - loss: 1.1102 - precision: 0.8241 - val_loss: 0.8414 - val_precision: 0.7928
Epoch 3/50
10917/10917 [==============================] - 1s 69us/sample - loss: 0.7900 - precision: 0.8160 - val_loss: 0.6891 - val_precision: 0.8293
Epoch 4/50
10917/10917 [==============================] - 1s 69us/sample - loss: 0.6950 - precision: 0.8349 - val_loss: 0.6190 - val_precision: 0.8859
Epoch 5/50
10917/10917 [==============================] - 1s 69us/sample - loss: 0.6145 - precision: 0.8638 - val_loss: 0.5434 - val_precision: 0.8849
Epoch 6/50
10917/10917 [==============================] - 1s 70us/sample - loss: 0.5868 - precision: 0.8708 - val_loss: 0.5208 - val_precision: 0.8901
Epoch 7/50
10917/10917 [====================

In [0]:
func.beeep()